In [9]:
import psycopg2
from psycopg2.extras import execute_values

conn = psycopg2.connect(
    dbname='testing',
    user='postgres',
    password='password',
    host='localhost',
    port='5432'
)
cur = conn.cursor()






query_add_media_type = """INSERT INTO media_type (name) VALUES (%s)"""

query_add_media_genre = """INSERT INTO media_genre (name) VALUES (%s)"""

query_add_media = """INSERT INTO media(name) VALUES (%s)"""

query_add_account = """
    INSERT INTO account(name, date_created, total_reviews)
    VALUES (%s, CURRENT_DATE, 0)"""

In [7]:
cur.execute("""
    DELETE from following;
    DELETE from review;
    DELETE from account;
""")
conn.commit()

In [17]:
#Should Be Done Before

genres = ['Horror', 'Fantasy', 'Action', 'Adventure', 'Romance', 'Documentary']
cur.executemany(query_add_media_type, [(type,) for type in types])

types = ['Movie', 'Tv-Show']
cur.executemany(query_add_media_genre, [(genre,) for genre in genres])

conn.commit()


In [21]:
#Here might be where batch upload from external source might occur

#First Movies (should be a superset for reviews to come)

cur.executemany(query_add_media, [(media,) for media in movies])
conn.commit()


#Then Reviews

...
...
...

In [11]:
#Sample Local Data (Created For Testing)


account_names = ['Venkat Gunturi', 'Gavin Heslip', 'Hashim Jama', 
                 'Zach Reid', 'Lucas Duncan', 'John Doe', 'Jane Smith',
                 'Kylie Tan', 'Bertha Blue', 'Tiffany Stoik']

cur.executemany(query_add_account, [(account,) for account in account_names])
conn.commit()







In [24]:


cur.execute(query_add_account, ("Lucas Duncan",))
cur.execute(query_add_account, ("John Doe",))
conn.commit()


In [2]:
query_get_media_id = """
    SELECT id from media where name = %s
"""

cur.execute(query_get_media_id, ("The Godfather",))

media_id = cur.fetchall()[0][0]

In [4]:
query_get_account_id = """ 
    SELECT id from account where name = %s
"""

cur.execute(query_get_account_id, ("Lucas Duncan",))
account_id = cur.fetchall()[0][0]

In [4]:
query_add_review = """
    INSERT INTO review(account_id, media_id, rating, description, date_reviewed)
    VALUES (%s, %s, %s, %s, CURRENT_DATE)
"""

cur.execute(query_add_review, (account_id, media_id, 8.5, "Good",))
conn.commit()

In [5]:
query_add_follower = """
    INSERT INTO following (account_id, follows_id, date_followed)
    VALUES (%s, %s, CURRENT_DATE);
"""

cur.execute(query_get_account_id, ("John Doe",))
account_id2 = cur.fetchall()[0][0]

cur.execute(query_add_follower, (account_id2, account_id))
conn.commit()


In [ ]:
query_follows = """
    SELECT u.id, u.name, u.total_reviews
    FROM following f
    JOIN account a on f.account_id = u.id
    WHERE f.follows_id = %s;
"""

query_following = """
    SELECT u.id, u.name, u.total_reviews 
    FROM following f
    JOIN account a ON f.follows_id = u.id
    WHERE f.account_id = %s;
"""

query_account_reviews = """
    SELECT r.id, r.rating, r.description
    FROM review r
    WHERE r.account_id = %s;
"""
query_genre_media_search_all = """
    SELECT m.id, m.name, AVG(r.rating) AS average_rating, COUNT(r.id) AS review_count
    FROM media m
    JOIN review r ON m.id = r.media_id
    WHERE m.genre = %s AND m.type = %s
    GROUP BY m.id, m.name
    ORDER BY average_rating DESC;
"""

query_genre_media_search_following = """
    SELECT m.id, m.name, AVG(r.rating) AS average_rating, COUNT(r.id) AS review_count
    FROM media m
    JOIN review r ON m.id = r.media_id
    JOIN media_genre g ON m.genre = g.id
    JOIN following f ON r.account_id = f.follows_id
    WHERE m.genre = %s AND m.type = %s AND f.account_id = %s
    GROUP BY m.id, m.name
    ORDER BY average_rating DESC;
"""

query_genre_media_search_compare = """
    SELECT m.id, m.name,
       AVG(CASE WHEN f.account_id IS NOT NULL THEN r.rating END) AS avg_rating_followed,
       AVG(CASE WHEN f.account_id IS NULL THEN r.rating END) AS avg_rating_not_followed
    FROM media m
    JOIN review r ON m.id = r.media_id
    JOIN media_genre g ON m.genre = g.id
    LEFT JOIN following f ON r.account_id = f.follows_id AND f.account_id = %s
    WHERE m.genre = %s AND m.type = %s
    GROUP BY m.id, m.name;
"""

query_see_recent_follower_reviews = """
    SELECT r.id, r.rating, r.description
    FROM review r
    JOIN account a ON r.account_id = u.id
    WHERE u.id IN (
        SELECT follows_id
        FROM following
        WHERE account_id = %s
        ORDER BY follows_id DESC
        LIMIT 20 
    );
"""